In [ ]:
!pip install pycsp3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 MB 8.2 MB/s eta 0:00:00


In [ ]:
from pycsp3 import *

*   **Modélisation du problème**







**1.   Les données du problème**



In [ ]:
""" Liste du nombre d’employés demandé par Amazon pour les 7 jour de la semaine."""
NE = [6,8,8,10,9,9,9]

""" * Liste à deux dimension représentant  les disponibilités de l'employés dans la semaine.
    * disp[i][j] == 1 si l'employé i est dispo le jour j sinon 0.
    * Voici une exemple à 14 employé qu'on va utiliser pour illustré le travaille.
"""
disp =  [[0,1,0,1,1,1,1],
         [0,0,1,1,1,1,1],
         [0,0,1,1,1,0,0],
         [1,1,1,1,1,1,0],
         [0,1,1,1,1,1,1],
         [0,1,1,1,1,1,1],
         [0,0,0,0,0,0,0],
         [1,0,0,0,0,0,1],
         [0,1,1,1,0,0,0],
         [1,0,0,1,1,1,1],
         [0,1,1,1,0,1,1],
         [1,0,0,1,1,1,1],
         [1,1,1,1,1,1,1],
         [1,1,1,1,1,0,0]]
"""  Quelque transoformation nécessaire pour la suite     """
tr_disp = list(map(list, zip(*disp)))
sum_disp = [sum(i) for i in disp]
sum_tr = [sum(i) for i in tr_disp]

**2.   Les variables de décision**

  Puisque Nous avont 7 x 14 variable de décision, J'utilise directement grâce au module pycsp un tableau de variable.

In [ ]:
" Création du tableau a len(disp) = 14 X len(NE) = 7   "
x = VarArray(size=[len(disp), len(NE)], dom=range(2))

**3.   Les contraintes du probléme**

*Contrainte 1 :*
\begin{equation}
      \sum\limits_{i\in E} x_{ij} \times disp(i,j) = ne(j), ~~\forall j\in J  
    \end{equation}

In [ ]:
def Constrainte1():
  satisfy(
          Sum(x[j,i]*tr_disp[i][j] for j in range(len(disp))) == NE[i]  for i in range(7)

  );

*Contrainte 2 :*
\begin{equation}
     \sum\limits_{i\in E} x_{ij} \leq \sum\limits_{i\in E} disp(i,j), ~~\forall j\in J
     \end{equation}


In [ ]:
def Contrainte_2():
  satisfy(
          Sum(x[:,i]) <= sum_tr[i] for i in range(7)
  );

*Contrainte 3:*
\begin{equation}
     \sum\limits_{j\in J} x_{ij} \leq \sum\limits_{j\in J} disp(i,j), ~~\forall i\in E
     \end{equation}

In [ ]:
def Contrainte_3():  
  satisfy(
          Sum(x[i]) <= sum(disp[i]) for i in range(len(disp))
    );

*Contrainte 4 : *

 A ) \begin{equation}
  \bigg|\sum\limits_{j\in J} x_{ij} - \sum\limits_{j\in J} x_{\tilde{i}j}\bigg| \leq S, ~~\forall (i,\tilde{i})\in E^2,~ i\neq \tilde{i}, ~ avec ~  \bigg|\sum\limits_{j\in J} disp(i,j)- \sum\limits_{j\in J} disp(\tilde{i},j)\bigg| \leq S  
\end{equation}


 B ) \begin{equation}
     \forall i\in E, \bigg(\sum\limits_{j\in J}disp(i,j) \leq \phi\bigg) \Longrightarrow \bigg(\sum\limits_{j\in J}x_{ij} = \sum\limits_{j\in J}disp(i,j)\bigg). 
\end{equation}
 
 avec : \begin{equation}
    \phi = \sum\limits_{j\in J}ne(j) / m 
\end{equation}


In [ ]:
"""      """
phi = sum(NE) // len([i  for i in  sum_disp if i !=0])

def Contrainte_4():
  satisfy(
        Sum(x[i]) == sum_disp[i]  if sum_disp[i] <= phi else  Sum(x[i]) >= phi for i in range(len(sum_disp))
        
   );
  satisfy(
    abs(Sum(x[i])- Sum(x[j])) <= 1 if abs(sum_disp[i]- sum_disp[j]) <= 1 and i != j else None for i in range(len(disp)) for j in range(len(disp)) 
    );  

*   **Résolution du problème**



In [ ]:
""" Cette fonction lorsque elle est appeler applique les contraintes, lance le solveur et affiche une solution si elle existe  """
def Resolution():
  Constrainte1()
  Contrainte_2()
  Contrainte_3()
  Contrainte_4()
  if solve(solver=ACE) is SAT:
    print(values(x))
  else:
    print("PAS DE SOLUTION")

In [ ]:
Resolution()

[
  [0, 1, 0, 1, 1, 1, 1]
  [0, 0, 1, 1, 1, 1, 1]
  [0, 0, 1, 1, 1, 0, 0]
  [1, 1, 1, 1, 1, 1, 0]
  [0, 1, 1, 0, 1, 1, 1]
  [0, 1, 0, 1, 1, 1, 1]
  [0, 0, 0, 0, 0, 0, 0]
  [1, 0, 0, 0, 0, 0, 1]
  [0, 1, 1, 1, 0, 0, 0]
  [1, 0, 0, 1, 1, 1, 1]
  [0, 1, 1, 1, 0, 1, 1]
  [1, 0, 0, 1, 1, 1, 1]
  [1, 1, 1, 0, 0, 1, 1]
  [1, 1, 1, 1, 1, 0, 0]
]


Dans cette partie est définit la méthode de la résolution incrémental présenté dans le rapport ou dans le cas ou on ne trouve pas de solution nous remplaçant la contrainte 1 par : 
\begin{equation}
      \sum\limits_{i\in E} x_{ij} \times disp(i,j) \leq ne(j), ~~\forall j\in J
    \end{equation}

In [ ]:
def Constrainte1_2():
  satisfy(
           Sum(x[j,i]*disp[j][i] for j in range(len(disp))) <= NE[i] for i in range(7)

  );

In [ ]:
def Inremental_Resolution():
  Contrainte_2()
  Contrainte_3()
  Contrainte_4()
  Constrainte1()
  if solve() is SAT:
    print(values(x))
  else:
    unpost()
    Constrainte1_2()
    if solve() is SAT:
      print(values(x))
    else:
        print("PAS DE SOLUTION")


In [ ]:
Inremental_Resolution()

[
  [0, 1, 0, 1, 1, 1, 1]
  [0, 0, 1, 1, 1, 1, 1]
  [0, 0, 1, 1, 1, 0, 0]
  [1, 1, 1, 1, 1, 1, 0]
  [0, 1, 1, 0, 1, 1, 1]
  [0, 1, 0, 1, 1, 1, 1]
  [0, 0, 0, 0, 0, 0, 0]
  [1, 0, 0, 0, 0, 0, 1]
  [0, 1, 1, 1, 0, 0, 0]
  [1, 0, 0, 1, 1, 1, 1]
  [0, 1, 1, 1, 0, 1, 1]
  [1, 0, 0, 1, 1, 1, 1]
  [1, 1, 1, 0, 0, 1, 1]
  [1, 1, 1, 1, 1, 0, 0]
]


*   **Partie Optimisation**

Comme mentionné dans le rapport, Un employé qui travaille sur deux ou plusieurs jours consécutifs va utiliser le même camion pour ses livraisons, c'est pour cela que idéalement, il est préférable qu'un employé soit affecté  sur des jours consécutifs, pour éviter qu'il ai à remettre le camion utilisé pour ses livraisons, à l'un de ses collègues planifié à travailler le lendemain.  
C’est pour cela que nous voudriont que la solution obtenu respecter le plus posible cette contrainte.

$$  \forall (i,l,k)\in J \ ,avec :  i≤ℓ≤k,  ~x_{ei}+x_{ek} \leq 1+x_{el}, \forall e\in E $$ 


In [ ]:
def Maximize_constrait():
  maximize(
      Sum(Sum(x[j,i]+x[j,k]) <= Sum(1+x[j,l]) if i<=l and l<=k else None for i in range(7) for l in range(7) for k in range(7) for j in range(len(disp)))
  );

In [ ]:
""" Dans ce block on affiche les solution optimal trouver """
def optimal_solutions():
  Constrainte1()
  Contrainte_2()
  Contrainte_3()
  Contrainte_4()
  Maximize_constrait()
  if solve(solver = ACE,sols = ALL) is  OPTIMUM:
      print("Number of solutions: ",n_solutions())
      for i in range(n_solutions()):
        print (f"Solution {i+1}")
        print(values(x,sol=i))


In [ ]:
optimal_solutions()


Number of solutions:  6
Solution 1
[
  [0, 1, 0, 1, 1, 1, 1]
  [0, 0, 1, 1, 1, 1, 1]
  [0, 0, 1, 1, 1, 0, 0]
  [1, 1, 1, 1, 0, 1, 0]
  [0, 1, 1, 0, 1, 1, 1]
  [0, 1, 0, 1, 1, 1, 1]
  [0, 0, 0, 0, 0, 0, 0]
  [1, 0, 0, 0, 0, 0, 1]
  [0, 1, 1, 1, 0, 0, 0]
  [1, 0, 0, 1, 1, 1, 1]
  [0, 1, 1, 1, 0, 1, 1]
  [1, 0, 0, 1, 1, 1, 1]
  [1, 1, 1, 0, 1, 1, 1]
  [1, 1, 1, 1, 1, 0, 0]
]
Solution 2
[
  [0, 1, 0, 1, 1, 1, 1]
  [0, 0, 1, 1, 1, 1, 1]
  [0, 0, 1, 1, 1, 0, 0]
  [1, 1, 1, 1, 1, 1, 0]
  [0, 1, 1, 0, 1, 1, 1]
  [0, 1, 0, 1, 1, 1, 1]
  [0, 0, 0, 0, 0, 0, 0]
  [1, 0, 0, 0, 0, 0, 1]
  [0, 1, 1, 1, 0, 0, 0]
  [1, 0, 0, 1, 1, 1, 1]
  [0, 1, 1, 1, 0, 1, 1]
  [1, 0, 1, 1, 0, 1, 1]
  [1, 1, 1, 0, 1, 1, 1]
  [1, 1, 1, 1, 1, 0, 0]
]
Solution 3
[
  [0, 1, 0, 1, 1, 1, 1]
  [0, 0, 1, 1, 1, 1, 1]
  [0, 0, 1, 1, 1, 0, 0]
  [1, 1, 1, 1, 1, 1, 0]
  [0, 1, 1, 0, 1, 1, 1]
  [0, 1, 0, 1, 1, 1, 1]
  [0, 0, 0, 0, 0, 0, 0]
  [1, 0, 0, 0, 0, 0, 1]
  [0, 1, 1, 1, 0, 0, 0]
  [1, 0, 0, 1, 1, 1, 1]
  [0, 1, 1, 1, 0, 1, 

Apres réflexion, j’ai estimé que la méthode de résolution incrémental est peu efficace surtout dans certain cas. C’est pour cela j’ai décider de remplacer les contraintes 1 définit par une seul contrainte conditionnel qui stipule que : Le nombre d’employés travaillant en une journée donnée doit etre égale au nombre d’employé nécessaires pour cette journée  si il lui est supérieur ou égal. Sinon, il est égale au nombre d’employé disponible pendant cette journée.

In [ ]:
def Constrainte1():
  satisfy(
    
          Sum(x[j,i]*tr_disp[i][j] for j in range(len(disp))) == NE[i] if NE[i] <= sum_tr[i] else Sum(x[j,i]*tr_disp[i][j] for j in range(len(disp))) == sum_tr[i]   for i in range(7)

  );


In [ ]:
optimal_solutions()

Number of solutions:  6
Solution 1
[
  [0, 1, 0, 1, 1, 1, 1]
  [0, 0, 1, 1, 1, 1, 1]
  [0, 0, 1, 1, 1, 0, 0]
  [1, 1, 1, 1, 0, 1, 0]
  [0, 1, 1, 0, 1, 1, 1]
  [0, 1, 0, 1, 1, 1, 1]
  [0, 0, 0, 0, 0, 0, 0]
  [1, 0, 0, 0, 0, 0, 1]
  [0, 1, 1, 1, 0, 0, 0]
  [1, 0, 0, 1, 1, 1, 1]
  [0, 1, 1, 1, 0, 1, 1]
  [1, 0, 0, 1, 1, 1, 1]
  [1, 1, 1, 0, 1, 1, 1]
  [1, 1, 1, 1, 1, 0, 0]
]
Solution 2
[
  [0, 1, 0, 1, 1, 1, 1]
  [0, 0, 1, 1, 1, 1, 1]
  [0, 0, 1, 1, 1, 0, 0]
  [1, 1, 1, 1, 1, 1, 0]
  [0, 1, 1, 0, 1, 1, 1]
  [0, 1, 0, 1, 1, 1, 1]
  [0, 0, 0, 0, 0, 0, 0]
  [1, 0, 0, 0, 0, 0, 1]
  [0, 1, 1, 1, 0, 0, 0]
  [1, 0, 0, 1, 1, 1, 1]
  [0, 1, 1, 1, 0, 1, 1]
  [1, 0, 1, 1, 0, 1, 1]
  [1, 1, 1, 0, 1, 1, 1]
  [1, 1, 1, 1, 1, 0, 0]
]
Solution 3
[
  [0, 1, 0, 1, 1, 1, 1]
  [0, 0, 1, 1, 1, 1, 1]
  [0, 0, 1, 1, 1, 0, 0]
  [1, 1, 1, 1, 1, 1, 0]
  [0, 1, 1, 0, 1, 1, 1]
  [0, 1, 0, 1, 1, 1, 1]
  [0, 0, 0, 0, 0, 0, 0]
  [1, 0, 0, 0, 0, 0, 1]
  [0, 1, 1, 1, 0, 0, 0]
  [1, 0, 0, 1, 1, 1, 1]
  [0, 1, 1, 1, 0, 1, 